# Bit set counts

* https://adventofcode.com/2021/day/3

We are tasked with counting bits in each column of input, to produce two binary numbers. While there are [some great algorithms for bit counting](https://graphics.stanford.edu/~seander/bithacks.html#CountBitsSetNaive), the fact that we are counting bits across columns means we need to use a slightly different approach.

The input, can be 'processed' by indexing into the column and testing if that column is equal to `'1'`, and summing these booleans (which in Python have integer values `0` and `1`) gives us the number of rows with the bit set. You can then compare that value with the number of lines and set the corresponding bit in gamma and epsilon (with epsilon being the inverse of gamma). By starting at the left-most column, and left-shifting the accumulated gamma and epsilon values, we naturally build up the bits in the correct order.

In [1]:
def power_consumption(report: list[str]) -> int:
    width = len(report[0])
    threshold = (len(report) + 1) // 2  # handles odd and even line lengths
    gamma = epsilon = 0
    for k in range(width):
        # The puzzle description just talks about 'most common'; it doesn't
        # matter to _my_ puzzle input if we break ties in favour of 1s or 0s.
        one_most_common = sum(line[k] == '1' for line in report) >= threshold
        gamma = gamma << 1 | one_most_common
        epsilon = epsilon << 1 | (not one_most_common)
    return gamma * epsilon


test_report = """\
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
""".splitlines()

assert power_consumption(test_report) == 198

In [2]:
import aocd
diagnostic_report = aocd.get_data(day=3, year=2021).splitlines(False)
print("Part 1:", power_consumption(diagnostic_report))

Part 1: 845186


# Part 2: Reducing the input numbers based on bit counts

Instead of accumulating a bit for each column in two numbers, we need to reduce the input lines based on the outcome. We'll have to assume that the puzzle inputs will always lead to a a single line out output for this process.

For each column index, calculate the threshold, determine if the `'1'` count is high enough, filter the lines based on the column value and and repeat until there is a single line left. Only then do we need to convert from string of binary digits to a number.

In [3]:
def life_support_rating(report: list[str]) -> int:
    width = len(report[0])
    
    def reduce(lines: list[str], filter_on='01') -> int:
        for k in range(width):
            threshold = (len(lines) + 1) // 2
            one_most_common = sum(line[k] == '1' for line in lines) >= threshold
            lines = [line for line in lines if line[k] == filter_on[one_most_common]]
            if len(lines) == 1:
                return int(lines[0], 2)
    
    oxygen_generator_rating = reduce(report)
    co2_scrubber_rating = reduce(report, filter_on="10")
    return oxygen_generator_rating * co2_scrubber_rating

assert life_support_rating(test_report) == 230

In [4]:
print("Part 2:", life_support_rating(diagnostic_report))

Part 2: 4636702
